Modèle sans convolution (réseau de neurones classique) sur le dataset du Joueur bas - Henri
- Coup droit
- Revers

In [22]:
import numpy as np
import cv2
import os
from smart.video import Video, Image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
import tensorflow as tf
import random
from sklearn.preprocessing import LabelEncoder

# Coup droit et revers

## Prises des frames et des vidéos

In [23]:
list_videos = []

In [24]:
y = []
all_paths = []
for dirpath, dirnames, _ in os.walk('img/JBas'):
    for dir_type in dirnames:
        
        path_fol = os.path.join(dirpath, dir_type)
        for file in os.listdir(path_fol):
            path_fol_img = os.path.join(path_fol, file)
            if os.path.isdir(path_fol_img) and file == 'images':
                frames = []
                for file_img in os.listdir(path_fol_img):
                    path_img = os.path.join(path_fol_img, file_img)
                    img_obj = Image.load_image(cv2.IMREAD_GRAYSCALE, path_img)
                    img = img_obj.img
                    frames.append(img)	
                if (len(frames) == 15):
                    output_res = path_img.split('\\')[1]
                    all_paths.append(path_fol_img)
                    vid = Video.read_video_from_frames(frames)
                    list_videos.append(vid)
                    y.append(output_res)

In [25]:
list_videos

In [26]:
# Output sans label encoder
y

['coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers']

In [27]:
# One-hot encoding
encoder = LabelEncoder()

Transformer output y en one hot vector
- 0: coup droit
- 1: revers

In [28]:
y = encoder.fit_transform(y)
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1], dtype=int64)

In [29]:
# Résolution des images
list_videos[0].frames[0].shape

(50, 50)

In [30]:
len(list_videos)

67

In [31]:
train_size = int(len(list_videos) * 0.8)
test_size = len(list_videos) - train_size

In [32]:
## Split video train, test
random.seed(42)
vids_train = random.sample(list_videos, k = train_size)
vids_test = [vid for vid in list_videos if vid not in vids_train]
y_train = [y[i] for i in range(len(y)) if list_videos[i] in vids_train]
y_test = [y[i] for i in range(len(y)) if list_videos[i] in vids_test]
paths_test = [all_paths[i] for i in range(len(all_paths)) if list_videos[i] in vids_test]
len(vids_train), len(y_train), len(vids_test), len(y_test)

(53, 53, 14, 14)

In [33]:
paths_test

['img/JBas\\coup droit\\car0001-0342253\\images',
 'img/JBas\\coup droit\\car5493-5847185\\images',
 'img/JBas\\coup droit\\na02011927\\images',
 'img/JBas\\coup droit\\na02012021\\images',
 'img/JBas\\coup droit\\na0201737\\images',
 'img/JBas\\coup droit\\uopc1606\\images',
 'img/JBas\\coup droit\\uopc424\\images',
 'img/JBas\\coup droit\\uopc698\\images',
 'img/JBas\\coup droit\\uopc955\\images',
 'img/JBas\\revers\\a36r977\\images',
 'img/JBas\\revers\\na02011122\\images',
 'img/JBas\\revers\\na02011195\\images',
 'img/JBas\\revers\\na0201493\\images',
 'img/JBas\\revers\\uopc1371\\images']

In [34]:
y_train = np.array(y_train, dtype=int)
y_test = np.array(y_test, dtype=int)
y_train

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1])

In [35]:
# Split input X en train, test set
X_train = np.zeros((train_size, 50*50*15), dtype=int)
for i, vid in enumerate(vids_train):
    seq_img = vid.frames.flatten()
    X_train[i] = seq_img
X_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [36]:
X_test = np.zeros((test_size, 50*50*15), dtype=int)
for i, vid in enumerate(vids_test):
    seq_img = vid.frames.flatten()
    X_test[i] = seq_img
X_test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

## Modèle de réseau de neurones avec classification binaires

In [37]:
tf.random.set_seed(1234) # for consistent results
model = Sequential(
    [               
        tf.keras.Input(shape = 50*50*15), # 50x50x15 = 37500
        Dense(units=16, activation="relu"),
        Dense(units=32, activation="relu"),
        Dense(units=128, activation="relu"),
        Dense(units=1, activation="sigmoid"),
    ]
)          
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 16)                600016    
                                                                 
 dense_6 (Dense)             (None, 32)                544       
                                                                 
 dense_7 (Dense)             (None, 128)               4224      
                                                                 
 dense_8 (Dense)             (None, 1)                 129       
                                                                 
Total params: 604,913
Trainable params: 604,913
Non-trainable params: 0
_________________________________________________________________


In [38]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [39]:
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = Adam(learning_rate=0.001), # 'Adam',
    metrics=['accuracy']
)

history = model.fit(
    X_train, y_train,
    epochs = 70,
    batch_size = 32, # données transmises pour une session
    validation_split=0.2,
    callbacks=callbacks_list
)

Epoch 1/70
1/2 [==============>...............] - ETA: 0s - loss: 7.0420 - accuracy: 0.6562
Epoch 1: val_loss improved from inf to 36.00470, saving model to Weights-001--36.00470.hdf5
2/2 [==============================] - 0s 120ms/step - loss: 14.7298 - accuracy: 0.6429 - val_loss: 36.0047 - val_accuracy: 0.0909
Epoch 2/70
1/2 [==============>...............] - ETA: 0s - loss: 18.7017 - accuracy: 0.5938
Epoch 2: val_loss improved from 36.00470 to 2.26896, saving model to Weights-002--2.26896.hdf5
2/2 [==============================] - 0s 37ms/step - loss: 15.8013 - accuracy: 0.6429 - val_loss: 2.2690 - val_accuracy: 0.5455
Epoch 3/70
1/2 [==============>...............] - ETA: 0s - loss: 4.4147 - accuracy: 0.5312
Epoch 3: val_loss did not improve from 2.26896
2/2 [==============================] - 0s 22ms/step - loss: 4.0073 - accuracy: 0.5952 - val_loss: 6.1398 - val_accuracy: 0.2727
Epoch 4/70
1/2 [==============>...............] - ETA: 0s - loss: 2.8896 - accuracy: 0.6875
Epoch 4: 

In [40]:
model.evaluate(X_test, y_test)

1/1 [==============================] - 0s 17ms/step - loss: 12.4700 - accuracy: 0.3571


[12.470030784606934, 0.3571428656578064]

In [41]:
pred = model.predict(X_test)
prediction_p = tf.nn.sigmoid(pred)
prediction_p

1/1 [==============================] - 0s 34ms/step


<tf.Tensor: shape=(14, 1), dtype=float32, numpy=
array([[0.7310586 ],
       [0.7310586 ],
       [0.7310551 ],
       [0.7310586 ],
       [0.5000059 ],
       [0.51469284],
       [0.7310586 ],
       [0.7310586 ],
       [0.5036303 ],
       [0.59707725],
       [0.7310586 ],
       [0.5004201 ],
       [0.7310586 ],
       [0.5       ]], dtype=float32)>

In [42]:
y_pred = tf.where(prediction_p < 0.5, 0, 1)
print("Predicted labels: ", y_pred)
print("True labels:      ", y_test)

Predicted labels:  tf.Tensor(
[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]], shape=(14, 1), dtype=int32)
True labels:       [0 0 0 0 0 0 0 0 0 1 1 1 1 1]


**Accuracy: $5/14$ ($35.71\%$)**

# Coup droit, revers, service et déplacement

## Prises des frames et des vidéos

In [3]:
list_videos = []
y = []
all_paths = []
for dirpath, dirnames, _ in os.walk('img/JBas'):
    for dir_type in dirnames:
        
        path_fol = os.path.join(dirpath, dir_type)
        for file in os.listdir(path_fol):
            path_fol_img = os.path.join(path_fol, file)
            if os.path.isdir(path_fol_img) and file == 'images':
                frames = []
                for file_img in os.listdir(path_fol_img):
                    path_img = os.path.join(path_fol_img, file_img)
                    img_obj = Image.load_image(cv2.IMREAD_GRAYSCALE, path_img)
                    img = img_obj.img
                    frames.append(img)	
                if (len(frames) == 15):
                    output_res = path_img.split('\\')[1]
                    all_paths.append(path_fol_img)
                    vid = Video.read_video_from_frames(frames)
                    list_videos.append(vid)
                    y.append(output_res)

In [4]:
# Output sans label encoder
y

['coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacem

Transformer output y en one hot vector
- 0: coup droit
- 1: déplacement
- 2: revers
- 3: service

In [8]:
y = encoder.fit_transform(y)
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
      dtype=int64)

In [9]:
output_y = np.unique(y)
output_y

array([0, 1, 2, 3], dtype=int64)

In [10]:
train_size = int(len(list_videos) * 0.8)
test_size = len(list_videos) - train_size

In [11]:
## Split video train, test
random.seed(42)
vids_train = random.sample(list_videos, k = train_size)
vids_test = [vid for vid in list_videos if vid not in vids_train]
y_train = [y[i] for i in range(len(y)) if list_videos[i] in vids_train]
y_test = [y[i] for i in range(len(y)) if list_videos[i] in vids_test]
paths_test = [all_paths[i] for i in range(len(all_paths)) if list_videos[i] in vids_test]
len(vids_train), len(y_train), len(vids_test), len(y_test)

(104, 104, 26, 26)

In [12]:
y_train = np.array(y_train, dtype=int)
y_test = np.array(y_test, dtype=int)

In [13]:
# Split input X en train, test set
X_train = np.zeros((train_size, 50*50*15), dtype=int)
for i, vid in enumerate(vids_train):
    seq_img = vid.frames.flatten()
    X_train[i] = seq_img
X_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [14]:
X_test = np.zeros((test_size, 50*50*15), dtype=int)
for i, vid in enumerate(vids_test):
    seq_img = vid.frames.flatten()
    X_test[i] = seq_img
X_test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

## Modèle de réseau de neurones avec classification multiclass

In [15]:
model_2 = Sequential(
    [               
        tf.keras.Input(shape = 50*50*15), # 50*50 * 15 = 37500
        Dense(units=16, activation="relu"),
        Dense(units=32, activation="relu"),
        Dense(units=64, activation="sigmoid"),
        Dense(units=128, activation="relu"),
        Dense(units=len(output_y), activation="softmax"),
    ]
)          
model_2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                600016    
                                                                 
 dense_1 (Dense)             (None, 32)                544       
                                                                 
 dense_2 (Dense)             (None, 64)                2112      
                                                                 
 dense_3 (Dense)             (None, 128)               8320      
                                                                 
 dense_4 (Dense)             (None, 4)                 516       
                                                                 
Total params: 611,508
Trainable params: 611,508
Non-trainable params: 0
_________________________________________________________________


In [16]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [17]:
model_2.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(), 
    optimizer = Adam(learning_rate=0.001),
    metrics=['accuracy']
)

history_2 = model_2.fit(
    X_train, y_train,
    epochs = 70,
    # batch_size = 32, # données transmises pour une session
    validation_split=0.2,
    callbacks=callbacks_list
)

Epoch 1/70
1/3 [=========>....................] - ETA: 0s - loss: 1.6367 - accuracy: 0.0625
Epoch 1: val_loss improved from inf to 1.51667, saving model to Weights-001--1.51667.hdf5
3/3 [==============================] - 0s 84ms/step - loss: 1.4726 - accuracy: 0.3012 - val_loss: 1.5167 - val_accuracy: 0.0000e+00
Epoch 2/70
1/3 [=========>....................] - ETA: 0s - loss: 1.2665 - accuracy: 0.5000
Epoch 2: val_loss did not improve from 1.51667
3/3 [==============================] - 0s 16ms/step - loss: 1.2259 - accuracy: 0.4819 - val_loss: 1.9074 - val_accuracy: 0.0000e+00
Epoch 3/70
1/3 [=========>....................] - ETA: 0s - loss: 1.1630 - accuracy: 0.4375
Epoch 3: val_loss did not improve from 1.51667
3/3 [==============================] - 0s 16ms/step - loss: 1.1311 - accuracy: 0.4699 - val_loss: 2.2622 - val_accuracy: 0.0000e+00
Epoch 4/70
1/3 [=========>....................] - ETA: 0s - loss: 1.0913 - accuracy: 0.5000
Epoch 4: val_loss did not improve from 1.51667
3/3 [

In [18]:
model_2.evaluate(X_test, y_test)

1/1 [==============================] - 0s 18ms/step - loss: 1.4615 - accuracy: 0.4615


[1.4615302085876465, 0.4615384638309479]

In [19]:
pred = model_2.predict(X_test)
pred

1/1 [==============================] - 0s 73ms/step


array([[1.49822980e-01, 8.17166984e-01, 3.23286019e-02, 6.81411300e-04],
       [2.14750364e-01, 7.28787780e-01, 5.56702837e-02, 7.91511906e-04],
       [2.14740381e-01, 7.28804529e-01, 5.56635968e-02, 7.91487517e-04],
       [6.83230758e-02, 9.18769181e-01, 1.24931317e-02, 4.14513663e-04],
       [4.06499356e-01, 7.23045915e-02, 5.20398974e-01, 7.97014509e-04],
       [6.83230758e-02, 9.18769181e-01, 1.24931317e-02, 4.14513663e-04],
       [2.14744627e-01, 7.28797436e-01, 5.56664653e-02, 7.91497878e-04],
       [5.93578160e-01, 3.06648493e-01, 9.89249200e-02, 8.48393945e-04],
       [1.29252151e-01, 8.47273707e-01, 2.28803810e-02, 5.93844743e-04],
       [2.14750364e-01, 7.28787780e-01, 5.56702837e-02, 7.91511906e-04],
       [1.93586066e-01, 5.96877158e-01, 2.08260536e-01, 1.27620203e-03],
       [5.94482303e-01, 3.13794374e-01, 9.04920027e-02, 1.23130088e-03],
       [2.14750364e-01, 7.28787780e-01, 5.56702837e-02, 7.91511906e-04],
       [2.09466517e-01, 7.38262713e-01, 5.14968038e

In [20]:
prediction = tf.nn.softmax(pred)
prediction

<tf.Tensor: shape=(26, 4), dtype=float32, numpy=
array([[0.21278197, 0.41472346, 0.18919411, 0.1833004 ],
       [0.23082228, 0.38594136, 0.19687478, 0.18636164],
       [0.2308193 , 0.3859467 , 0.19687288, 0.1863611 ],
       [0.19154385, 0.44834492, 0.181143  , 0.17896819],
       [0.28546453, 0.20436813, 0.31990284, 0.19026454],
       [0.19154385, 0.44834492, 0.181143  , 0.17896819],
       [0.23082054, 0.3859444 , 0.1968737 , 0.18636133],
       [0.34326947, 0.25764588, 0.20931965, 0.1897649 ],
       [0.20709348, 0.4246188 , 0.18619575, 0.182092  ],
       [0.23082228, 0.38594136, 0.19687478, 0.18636164],
       [0.23059666, 0.34514382, 0.23400551, 0.19025405],
       [0.34341753, 0.2593709 , 0.20746376, 0.18974783],
       [0.23082228, 0.38594136, 0.19687478, 0.18636164],
       [0.22923116, 0.38897967, 0.19573489, 0.18605426],
       [0.19154385, 0.44834492, 0.181143  , 0.17896819],
       [0.23082228, 0.38594136, 0.19687478, 0.18636164],
       [0.26675823, 0.3021463 , 0.23885

In [21]:
y_pred = np.argmax(prediction, axis=1)
print("Predicted labels: ", y_pred)
print("True labels:      ", y_test)

Predicted labels:  [1 1 1 1 2 1 1 0 1 1 1 0 1 1 1 1 1 2 1 1 1 1 0 0 1 1]
True labels:       [0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 3]
